# Introduction
Connect Four (or Connect - 4) is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs. Connect Four is a solved game. The first player can always win by playing the right moves. If the board fills up before either player achieves four in a row, then the game is a draw.

# Objective
To build a machine learning model to find whether first player has won the game or not with given connect-4 positions.

# ML Problem:
Supervised Learning - Classification

# Metrics:
F1 -score

# Approach 
1. Building a CNN Model.(Board looks like 6*7 Image Pixel.So my intial though was building a CNN Model helps to preserve spatial information)



In [ ]:
# importing library
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.optimizers import Adam
from keras.utils import np_utils

In [ ]:
seed=5

In [ ]:
# import data
train_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/connect-4/train_set_label.csv")
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/connect-4/test_set_label.csv')

In [ ]:
# Train data - first 5 rows
train_data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,b,b,b,b,b,b,o,o,x,b,b,b,x,x,o,x,b,b,b,b,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,loss
1,b,b,b,b,b,b,o,b,b,b,b,b,x,o,o,b,b,b,x,b,b,b,b,b,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,loss
2,x,x,o,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,o,x,x,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,win
3,x,x,b,b,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,b,b,b,b,b,o,x,o,x,b,b,win
4,x,x,o,o,b,b,o,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,o,x,x,b,b,b,b,b,b,b,b,b,win


In [ ]:
print("Train Data :",train_data.shape[0],"rows",train_data.shape[1],"Columns")
print("Train Data :",test_data.shape[0],"rows",test_data.shape[1],"Columns")

Train Data : 50667 rows 43 Columns
Train Data : 16890 rows 42 Columns


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50667 entries, 0 to 50666
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a1      50667 non-null  object
 1   a2      50667 non-null  object
 2   a3      50667 non-null  object
 3   a4      50667 non-null  object
 4   a5      50667 non-null  object
 5   a6      50667 non-null  object
 6   b1      50667 non-null  object
 7   b2      50667 non-null  object
 8   b3      50667 non-null  object
 9   b4      50667 non-null  object
 10  b5      50667 non-null  object
 11  b6      50667 non-null  object
 12  c1      50667 non-null  object
 13  c2      50667 non-null  object
 14  c3      50667 non-null  object
 15  c4      50667 non-null  object
 16  c5      50667 non-null  object
 17  c6      50667 non-null  object
 18  d1      50667 non-null  object
 19  d2      50667 non-null  object
 20  d3      50667 non-null  object
 21  d4      50667 non-null  object
 22  d5      50667 non-null

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16890 entries, 0 to 16889
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   a1      16890 non-null  object
 1   a2      16890 non-null  object
 2   a3      16890 non-null  object
 3   a4      16890 non-null  object
 4   a5      16890 non-null  object
 5   a6      16890 non-null  object
 6   b1      16890 non-null  object
 7   b2      16890 non-null  object
 8   b3      16890 non-null  object
 9   b4      16890 non-null  object
 10  b5      16890 non-null  object
 11  b6      16890 non-null  object
 12  c1      16890 non-null  object
 13  c2      16890 non-null  object
 14  c3      16890 non-null  object
 15  c4      16890 non-null  object
 16  c5      16890 non-null  object
 17  c6      16890 non-null  object
 18  d1      16890 non-null  object
 19  d2      16890 non-null  object
 20  d3      16890 non-null  object
 21  d4      16890 non-null  object
 22  d5      16890 non-null

In [ ]:
train_data.duplicated().sum()

0

In [ ]:
test_data.duplicated().sum()

0

In [ ]:
train_data['class'].value_counts(normalize=True)

win     0.658298
loss    0.246235
draw    0.095466
Name: class, dtype: float64

* No Null Values.
* All Columns are of 'Object' Datatype.
* No Duplicate Values.
* Dataset is Unbalanced.

In [ ]:
# Label Encoding
train_data['class'] = train_data['class'].map({"win":0,"loss":1,"draw":2})

# Seperating Target variable and Independent Variable
X = train_data.drop(['class'],axis=1)
y = train_data[['class']]

In [ ]:
# Label encoding
for b in X.columns:
  X[b] = X[b].map({'x':-1,'b':0,'o':1})
  test_data[b] = test_data[b].map({'x':-1,'b':0,'o':1})

In [ ]:
# Splitting given data to training data and Validation data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,stratify=y,random_state = seed)

In [ ]:
# Train Data & Validation Data a
display("Train Data :",X_train.shape[0],"rows",X_train.shape[1],"Columns")
display("Validation Data :",X_test.shape[0],"rows",X_test.shape[1],"Columns")

'Train Data :'

45600

'rows'

42

'Columns'

'Validation Data :'

5067

'rows'

42

'Columns'

In [ ]:
# Converting to Numpy array
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [ ]:
# Reshaping the array to 4 dimension
X_train = X_train.reshape(-1,7,6,1)
X_test = X_test.reshape(-1,7,6,1)

In [ ]:
X_train[0]

array([[[ 1],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]],

       [[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]],

       [[-1],
        [-1],
        [ 1],
        [-1],
        [ 0],
        [ 0]],

       [[-1],
        [ 1],
        [ 1],
        [ 0],
        [ 0],
        [ 0]],

       [[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]],

       [[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]],

       [[ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]])

In [ ]:
"""import cv2
from google.colab.patches import cv2_imshow
cv2_imshow(X_train[3])"""

'import cv2\nfrom google.colab.patches import cv2_imshow\ncv2_imshow(X_train[3])'

In [ ]:
# One hot encoding on target variable
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)

In [ ]:
y_train[0]

array([1., 0., 0.], dtype=float32)

In [ ]:
y_train.shape

(45600, 3)

In [ ]:
y_test.shape

(5067, 3)

In [ ]:
# CNN Model
model = Sequential()
model.add(Conv2D(128,kernel_size=(7,6),activation='relu',input_shape=(7,6,1),padding='same'))
#model.add(MaxPool2D((2,2)))
#model.add(Dropout(0.3))
model.add(Conv2D(128,kernel_size=(7,6),activation='relu',padding='same'))
#model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [ ]:
# model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 6, 128)         5504      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 6, 128)         688256    
_________________________________________________________________
flatten (Flatten)            (None, 5376)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               688256    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 1,390,467
Trainable params: 1,390,467
Non-trainable params: 0
______________________________________________

In [ ]:
# Compiling Model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Fitting Model
epochs = 20
batch_size = 32
history = model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test,y_test))

Epoch 1/20
1425/1425 [==============================] - 209s 146ms/step - loss: 0.5858 - accuracy: 0.7585 - val_loss: 0.3958 - val_accuracy: 0.8417
Epoch 2/20
1425/1425 [==============================] - 207s 145ms/step - loss: 0.3890 - accuracy: 0.8407 - val_loss: 0.3662 - val_accuracy: 0.8542
Epoch 3/20
1425/1425 [==============================] - 207s 145ms/step - loss: 0.3285 - accuracy: 0.8664 - val_loss: 0.3563 - val_accuracy: 0.8555
Epoch 4/20
1425/1425 [==============================] - 208s 146ms/step - loss: 0.2858 - accuracy: 0.8834 - val_loss: 0.3350 - val_accuracy: 0.8674
Epoch 5/20
1425/1425 [==============================] - 208s 146ms/step - loss: 0.2409 - accuracy: 0.9019 - val_loss: 0.3564 - val_accuracy: 0.8699
Epoch 6/20
1425/1425 [==============================] - 208s 146ms/step - loss: 0.2007 - accuracy: 0.9191 - val_loss: 0.3506 - val_accuracy: 0.8680
Epoch 7/20
1425/1425 [==============================] - 208s 146ms/step - loss: 0.1662 - accuracy: 0.9350 - val_

In [ ]:
# converting test data to Numpy array
test_data = test_data.to_numpy()
# reshaping the array
test_data = test_data.reshape(-1,7,6,1)

In [ ]:
# Predicting probability of the target class
y_predict = model.predict(test_data)

In [ ]:
y_predict

array([[6.77230751e-15, 1.00000000e+00, 3.85213583e-10],
       [1.00000000e+00, 3.15366087e-14, 2.59522806e-15],
       [1.00000000e+00, 5.47578445e-17, 1.69187130e-11],
       ...,
       [1.00000000e+00, 1.80718076e-27, 1.10242886e-29],
       [9.99997497e-01, 1.40782448e-16, 2.52048176e-06],
       [9.99999762e-01, 1.52999544e-10, 1.82561223e-07]], dtype=float32)

In [ ]:
# variable that has high probability
results = np.argmax(y_predict,axis = 1)

In [ ]:
# reversing label encoding
results = pd.Series(results,name="prediction")
results = results.map({0:"win",1:"loss",2:"draw"})

In [ ]:
results

0        loss
1         win
2         win
3         win
4         win
         ... 
16885     win
16886     win
16887     win
16888     win
16889     win
Name: prediction, Length: 16890, dtype: object

In [ ]:
# extracting pdf as csv file
results.to_csv("connect4.csv",index=False)